<a href="https://colab.research.google.com/github/Sidharth-Saji/WildFire_Monitoring_Using_Satellite_Image/blob/main/Model_Training_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations as A

c:\Users\neems\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.2 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


## Creating Dataframes

In [2]:
def create_df_img(filepath):
    """ Create a DataFrame with the filepath and the labels of the pictures
    
    Args : filepath of the images
    Return : pandas dataframe     
    """
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    df = pd.concat([filepath, labels], axis=1)
    df = df.sample(frac=1).reset_index(drop = True)
    return df

In [3]:
train_dir = Path(r'E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver\wildfire_detection\datasets\kaggle\prediction\train')
filepaths = list(train_dir.glob(r'**/*.jpg'))
val_dir = Path(r'E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver\wildfire_detection\datasets\kaggle\prediction\valid')
filepaths_valid = list(val_dir.glob(r'**/*.jpg'))

### Checking if dataframe loaded correctly

In [4]:
df_train = create_df_img(filepaths)
df_valid = create_df_img(filepaths_valid)
df_train.head()
df_valid.head()

,Filepath,Label
0,E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver...,wildfire
1,E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver...,nowildfire
2,E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver...,wildfire
3,E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver...,wildfire
4,E:\Programs\TSEC\TSEC-HACKS---artemis\mlserver...,wildfire


#DATA AUGMENTATION 

In [5]:
def augmentation_image(image):
    """
    Function for Augmenting Images
    """
    aug = A.Compose([
        A.Flip(),
        A.Rotate(limit=75,always_apply=True),
    ])
    augmented_image = aug(image=image)['image']
    return augmented_image

In [6]:
BATCH_SIZE = 256
# Creating Train Generator using Image Generator
train_datagenerator = ImageDataGenerator(dtype='float32', rescale= 1./255., preprocessing_function=augmentation_image)
train_generator = train_datagenerator.flow_from_dataframe(df_train,
                                                    x_col='Filepath',
                                                    y_col='Label',
                                                    shuffle=True,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = (350,350),
                                                    class_mode = 'categorical')

#Creating Validity Checker
val_datagen = ImageDataGenerator(dtype='float32', rescale= 1./255.)
val_generator = val_datagen.flow_from_dataframe(df_valid,
                                                    x_col='Filepath',
                                                    y_col='Label',
                                                    shuffle=True,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = (350,350),
                                                    class_mode = 'categorical')

Found 30250 validated image filenames belonging to 2 classes.
Found 6300 validated image filenames belonging to 2 classes.


# MODEL BUILDING

In [7]:
shrink = 1e-3 
image_shape=plt.imread(df_train["Filepath"][0]).shape
model = Sequential([
    Conv2D(filters = 8 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 16 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 32 , kernel_size = 2, activation = 'relu',
           kernel_regularizer = regularizers.l2(shrink)),
    MaxPooling2D(pool_size = 2),
    
    
    Flatten(),
    Dense(300,activation='relu'),
    Dropout(0.5),
    Dense(100,activation='relu'),
    Dropout(0.4),
    Dense(2,activation='softmax')
])
model.summary()

c:\Users\neems\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 349, 349, 8)    │           104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 174, 174, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 173, 173, 16)   │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 86, 86, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 85, 85, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 42, 42, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 56448)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │    16,934,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,967,714 (64.73 MB)

 Trainable params: 16,967,714 (64.73 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

checkpointer = ModelCheckpoint('model.keras', verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

optimizer = optimizers.Adam(learning_rate=0.001, weight_decay=1e-5)

model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['AUC', 'acc'])

history = model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=val_generator,
                    callbacks=[checkpointer, early_stopping])

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - AUC: 0.8157 - acc: 0.7489 - loss: 0.5622
Epoch 1: val_loss improved from inf to 0.19321, saving model to model.keras
119/119 ━━━━━━━━━━━━━━━━━━━━ 576s 5s/step - AUC: 0.8166 - acc: 0.7498 - loss: 0.5606 - val_AUC: 0.9795 - val_acc: 0.9354 - val_loss: 0.1932
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - AUC: 0.9726 - acc: 0.9236 - loss: 0.2214
Epoch 2: val_loss improved from 0.19321 to 0.17056, saving model to model.keras
119/119 ━━━━━━━━━━━━━━━━━━━━ 530s 4s/step - AUC: 0.9726 - acc: 0.9236 - loss: 0.2214 - val_AUC: 0.9835 - val_acc: 0.9421 - val_loss: 0.1706
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - AUC: 0.9746 - acc: 0.9265 - loss: 0.2125
Epoch 3: val_loss improved from 0.17056 to 0.16046, saving model to model.keras
119/119 ━━━━━━━━━━━━━━━━━━━━ 364s 3s/step - AUC: 0.9746 - acc: 0.9265 - loss: 0.2124 - val_AUC: 0.9857 - val_acc: 0.9448 - val_loss: 0.1605
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - AUC: 0.98

In [11]:
model.save('WildFireDetector.h5')